In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import sys
import p5lib

%matplotlib inline

In [2]:
data = pd.read_csv('MERGED2013_PP.csv')
print "Number of features: {}".format(len(data.columns))
print "Number of rows: {}".format(len(data))

Number of features: 1729
Number of rows: 7804


/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data = p5lib.preprocess_data(data)
print data.shape

(6007, 42)


In [48]:

X = data[['CONTROL', 'DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'PAR_ED_PCT_1STGEN', 
          'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv', 'UGDS', 
          'WDRAW_DEBT_MDN', 'L4_COLLEGE', 'NPT4', 'NUM4', 
          'PFTFTUG1_EF', 'PFTFAC',
          'SAT_AVG_ALL', 'ACTCMMID', 'ADM_RATE_ALL', 'AVGFACSAL', 'COSTT4_A',
          'CCSIZSET', 
          'CCUGPROF', 
          'CCBASIC',
          'C150', 'RET_FT']]
print X.shape
# print "after cleaning for outliers"
# X = X[~((X.C150 == 1) & (X.RET_FT < 0.5))]
# X = X[~((X.RET_FT == 1) & (X.C150 < 0.5))]
# print X.shape

(6007, 30)


In [49]:
fill_cols_with_mean = ['DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'WDRAW_DEBT_MDN',
                       'PAR_ED_PCT_1STGEN', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 
                       'UG25abv', 'NPT4', 'NUM4', 'PFTFTUG1_EF', 'PFTFAC', 'SAT_AVG_ALL', 'ACTCMMID', 'ADM_RATE_ALL', 
                       'AVGFACSAL', 'COSTT4_A']
for col in fill_cols_with_mean:
    X[col] = X[col].fillna(X[col].mean())

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [50]:
X = X.dropna()

y = X[['C150', 'RET_FT']]
X = X.drop('C150', 1)
X = X.drop('RET_FT', 1)

print X.shape
print y.shape


(3286, 28)
(3286, 2)


In [51]:
from sklearn import preprocessing

tmpX = X[['DEBT_MDN', 'DEP_INC_AVG', 'GRAD_DEBT_MDN', 'IND_INC_AVG', 'INEXPFTE', 'UGDS', 'WDRAW_DEBT_MDN', 'NPT4', 
          'NUM4', 'SAT_AVG_ALL', 'ACTCMMID', 'AVGFACSAL', 'COSTT4_A']]

scaler = preprocessing.MinMaxScaler()
scaledX = scaler.fit_transform(tmpX)

pctX = X[['PAR_ED_PCT_1STGEN', 'PAR_ED_PCT_HS', 'PAR_ED_PCT_MS', 'PAR_ED_PCT_PS', 'PCTFLOAN', 'PCTPELL', 'UG25abv', 
          'PFTFTUG1_EF', 'PFTFAC', 'ADM_RATE_ALL']]
pctX = np.array(pctX)

forPcaX = np.concatenate((scaledX, pctX), axis=1)
# print scaledX.shape
# print pctX.shape
print forPcaX.shape


(3286, 23)


In [52]:
from sklearn.decomposition import PCA

pca = PCA(n_components=forPcaX.shape[1])
pca.fit(forPcaX)

print pca.explained_variance_ratio_


[  3.73856482e-01   1.95353249e-01   1.21146530e-01   7.25973650e-02
   5.33455066e-02   3.46573825e-02   3.14777787e-02   2.38411287e-02
   2.04015592e-02   1.55729709e-02   1.29169933e-02   1.01393362e-02
   8.04262430e-03   7.27408208e-03   5.91667125e-03   4.26327010e-03
   3.81588791e-03   2.26469323e-03   1.66358417e-03   1.08855908e-03
   3.49301605e-04   1.50286240e-05   1.61424858e-08]


In [53]:
pca = PCA(n_components=18)
pca.fit(forPcaX)
reducedX = pca.transform(forPcaX)
print reducedX.shape

(3286, 18)


In [54]:
# add the categorical feature

tmp1X = np.array(pd.get_dummies(X['CONTROL']))
tmp2X = X['L4_COLLEGE'].astype(int)
tmp2X = np.reshape(tmp2X, (len(tmp2X), 1)) 
tmp3X = np.array(pd.get_dummies(X['CCSIZSET']))
tmp4X = np.array(pd.get_dummies(X['CCUGPROF']))
tmp5X = np.array(pd.get_dummies(X['CCBASIC']))

print tmp1X.shape
print tmp2X.shape
print reducedX.shape
finalX = np.concatenate((reducedX, tmp1X, tmp2X, tmp3X, tmp4X), axis=1)
print finalX.shape

(3286, 3)
(3286, 1)
(3286, 18)
(3286, 53)


In [55]:
from sklearn import cross_validation as cv

y = np.array(y)
X_train, X_test, y_train, y_test = cv.train_test_split(finalX, y, train_size=0.8)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(2628, 53)
(658, 53)
(2628, 2)
(658, 2)


In [56]:
SVR_reg1, SVR_reg2 = p5lib.build_SVR_model(X_train, X_test, y_train, y_test)

{'epsilon': 0.10000000000000001, 'C': 10.0, 'gamma': 0.10000000000000001}
R2 score on train data: 0.859977441103
R2 score on test  data: 0.741611996931
{'epsilon': 0.10000000000000001, 'C': 0.10000000000000001, 'gamma': 0.10000000000000001}
R2 score on train data: 0.507539696576
R2 score on test  data: 0.42246816718


In [42]:
DT_reg1, DT_reg2 = p5lib.build_DecisionTree_model(X_train, X_test, y_train, y_test)

{'max_depth': 5}
R2 score on train data: 0.72984240074
R2 score on test  data: 0.645041563667
{'max_depth': 4}
R2 score on train data: 0.473773070302
R2 score on test  data: 0.435589696255


In [43]:
KNN_reg1, KNN_reg2 = p5lib.build_KNN_model(X_train, X_test, y_train, y_test)

{'n_neighbors': 7}
R2 score on train data: 0.788018342205
R2 score on test  data: 0.690703553646
{'n_neighbors': 18}
R2 score on train data: 0.523614080128
R2 score on test  data: 0.487362501832


In [44]:
RForest_reg1, RForest_reg2 = p5lib.build_RandomForest_model(X_train, X_test, y_train, y_test)

R2 score on train data: 0.952834932729
R2 score on test  data: 0.714627768305
R2 score on train data: 0.913098164092
R2 score on test  data: 0.493897008711


In [45]:
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV

scorer = p5lib.scorer

y1_train, y1_test, y2_train, y2_test = p5lib.split_y(y_train, y_test)

params = {'C': np.logspace(-1, 1, 2), 'gamma': np.logspace(-1, 1, 2), 'epsilon': np.logspace(-1, 1, 2)}
reg = SVR()
best_reg1 = GridSearchCV(reg, params, scoring=scorer, cv=5)
best_reg1.fit(X_train, y1_train)

print best_reg1.best_params_
p5lib.print_r2score(best_reg1, X_train, y1_train)
p5lib.print_r2score(best_reg1, X_test, y1_test, test=True)

reg = SVR()
params = {'C': np.logspace(-1, 1, 2), 'gamma': np.logspace(-1, 1, 2), 'epsilon': np.logspace(-1, 1, 2)}
best_reg2 = GridSearchCV(reg, params, scoring=scorer, cv=5)

# use the y1 data to train the y2 prediction
reshaped_y1_train = np.reshape(y1_train, (len(y1_train), 1))
X_train_boosted = np.concatenate((X_train, reshaped_y1_train), axis=1)
best_reg2.fit(X_train_boosted, y2_train)

print best_reg2.best_params_

y1_test_prediction = best_reg1.predict(X_test) # prediction of y1 from the first model
y1_test_prediction = np.reshape(y1_test_prediction, (len(y1_test_prediction), 1))
X_test_boosted = np.concatenate((X_test, y1_test_prediction), axis=1)
p5lib.print_r2score(best_reg2, X_train_boosted, y2_train)
p5lib.print_r2score(best_reg2, X_test_boosted, y2_test, test=True)

{'epsilon': 0.10000000000000001, 'C': 10.0, 'gamma': 0.10000000000000001}
R2 score on train data: 0.862237951907
R2 score on test  data: 0.756320716499
{'epsilon': 0.10000000000000001, 'C': 0.10000000000000001, 'gamma': 0.10000000000000001}
R2 score on train data: 0.591833821973
R2 score on test  data: 0.536095914962


In [47]:
from sklearn.ensemble import RandomForestRegressor

scorer = p5lib.scorer

y1_train, y1_test, y2_train, y2_test = p5lib.split_y(y_train, y_test)


reg1 = RandomForestRegressor(n_estimators=10)
reg1.fit(X_train, y1_train)
p5lib.print_r2score(reg1, X_train, y1_train)
p5lib.print_r2score(reg1, X_test, y1_test, test=True)

reg2 = RandomForestRegressor(n_estimators=10)
# reg2.fit(X_train, y2_train)
# print_r2score(reg2, X_train, y2_train)
# print_r2score(reg2, X_test, y2_test, test=True)

# use the y1 data to train the y2 prediction
reshaped_y1_train = np.reshape(y1_train, (len(y1_train), 1))
X_train_boosted = np.concatenate((X_train, reshaped_y1_train), axis=1)
reg2.fit(X_train_boosted, y2_train)

y1_test_prediction = reg1.predict(X_test) # prediction of y1 from the first model
y1_test_prediction = np.reshape(y1_test_prediction, (len(y1_test_prediction), 1))
X_test_boosted = np.concatenate((X_test, y1_test_prediction), axis=1)
p5lib.print_r2score(reg2, X_train_boosted, y2_train)
p5lib.print_r2score(reg2, X_test_boosted, y2_test, test=True)

R2 score on train data: 0.957074726135
R2 score on test  data: 0.676911004759
R2 score on train data: 0.915598691456
R2 score on test  data: 0.478273404464
